### This notebook contains the code for data preprocessing before training and analysis 

In [1]:
import pandas as pd
import numpy as np
import pickle
import re
from os import listdir
import os
import sys
import re
import pandas as pd
import numpy as np

# importing library for stoichometric calculation 
sys.path.insert(0, "/scripts")
from utils_chalcogenides import *


In [2]:
# components list acceptable for chalcogenide 
import csv
compounds = []
with open('com_names.csv') as csv_file:
    reader = csv.reader(csv_file, delimiter=',')
    rows = list(reader)
    
    for i in (rows):
        compounds.append(i[0])
# compounds.append('Temp')


In [4]:

df1 = pd.read_csv('/molar/molar_Shear_modulus_0050.csv')
df2 = pd.read_csv('/atomic/atomic_Shear_modulus_0050.csv')
df4 = pd.read_csv('/sg/shear_modulus_sg.csv')


In [5]:
# renaming and unit handling 
prop_ = "Shear_modulus"
# df1 = df1.rename(columns={"PROP_FIG_SI":prop_})
df1 = df1.rename(columns={"PROP_FIG_SI":prop_})
df2 = df2.rename(columns={"PROP_FIG_SI":prop_})
df4 = df4.rename(columns={"MOD_SDV":prop_})

df1[prop_] /= 10**9
df2[prop_] /= 10**9



In [6]:
df2

,O,Si,Bi,Er,Na,Ba,Mg,Al,K,Ca,...,Tl,C,D,Lu,Ho,Hf,At,GLASS_ID,PROP_CONDITION,Shear_modulus
0,44.44,0.0,0.0,0.0,38.89,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3226,NaN,NaN
1,57.89,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3944,NaN,24.49
2,57.89,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3945,NaN,27.02
3,57.89,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3946,NaN,25.75
4,57.89,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3947,NaN,24.57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397,65.89,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10184,NaN,41.12
398,64.86,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10185,NaN,43.12
399,57.53,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10186,NaN,38.85
400,57.41,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10189,NaN,31.74


In [7]:
# concating 
df = pd.concat([df1, df2, df4] , sort = False)
col_list = df.columns.tolist()  
col_list.remove(prop_)
col_list = col_list + [prop_]
df = df[col_list]

In [8]:
# processing 
a = df[prop_]
df.drop(columns=[col for col in df if col not in compounds], inplace=True)
df.insert(len(df.columns), prop_, a)
df = df.fillna(0)

In [ ]:
# removing duplicate and stoichometric calculation, see chalcogenide_utils.py
features = df.columns.values.tolist()
features.remove(prop_)
df = remove_duplicate(df)
d = df.to_dict(orient='records')
final_dict , chem = create_chemicalDict(features, prop_)
df = data_creator(d,final_dict, chem, prop_)
df = remove_minSamples(df , 10 )

In [274]:
df.to_csv('Shear_modulus.csv', index=False)
